In [1]:
# 📌 Import bibliotek
import os
from collections import Counter
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

In [2]:
# 🗂️ Ścieżki do danych (zmień, jeśli masz inną)
root_dir = "./real-and-fake-detection/real_vs_fake/real-vs-fake"

In [3]:
# Funkcja do przejrzenia katalogów
def get_dataset_info(root):
    info = {}
    for split in ["train", "valid", "test"]:
        split_path = os.path.join(root, split)
        if not os.path.isdir(split_path):
            continue

        classes = {}
        for cls in os.listdir(split_path):
            cls_path = os.path.join(split_path, cls)
            if os.path.isdir(cls_path):
                classes[cls] = [
                    os.path.join(cls_path, f)
                    for f in os.listdir(cls_path)
                    if f.lower().endswith((".png", ".jpg", ".jpeg"))
                ]
        info[split] = classes
    return info

In [4]:
# 📊 Zebranie statystyk
dataset_info = get_dataset_info(root_dir)

print("🔍 Podstawowa struktura datasetu:")
for split, classes in dataset_info.items():
    print(f"\n📦 {split.upper()}:")
    for cls, files in classes.items():
        print(f" - {cls}: {len(files):,} obrazów")

🔍 Podstawowa struktura datasetu:

📦 TRAIN:
 - fake: 50,000 obrazów
 - real: 50,000 obrazów

📦 VALID:
 - fake: 10,000 obrazów
 - real: 10,000 obrazów

📦 TEST:
 - fake: 10,000 obrazów
 - real: 10,000 obrazów


In [ ]:
# 🖼️ Wizualne sprawdzenie przykładowych obrazów
def display_samples(info, n=3):
    fig, axes = plt.subplots(len(info), n, figsize=(4*n, 4*len(info)))
    if len(info) == 1:
        axes = [axes]

    for i, (split, classes) in enumerate(info.items()):
        img_paths = []
        for cls, files in classes.items():
            img_paths += files[:n]

        for j, img_path in enumerate(img_paths):
            try:
                img = Image.open(img_path)
                axes[i][j].imshow(img)
                axes[i][j].axis("off")
                axes[i][j].set_title(f"{split} — {os.path.basename(os.path.dirname(img_path))}")
            except Exception as e:
                print(f"⚠️ Błąd wczytywania {img_path}: {e}")
    plt.show()

print("\n📷 Przykładowe obrazy z datasetu:")
display_samples(dataset_info)

In [5]:
# 🧠 Parametry obrazów: rozmiary i kanały
def analyze_image_properties(file_list, n_samples=100):
    dims = []
    color_modes = Counter()
    for fp in np.random.choice(file_list, min(n_samples, len(file_list)), replace=False):
        try:
            img = Image.open(fp)
            dims.append(img.size)  # (width, height)
            color_modes[img.mode] += 1
        except:
            pass

    return dims, color_modes

all_files = sum([files for split in dataset_info.values() for files in split.values()], [])

dims, modes = analyze_image_properties(all_files)
print("\n⚙️ Przykładowe rozmiary obrazów (pierwsze 10):", dims[:10])
print("📊 Tryby kolorów:", modes)


⚙️ Przykładowe rozmiary obrazów (pierwsze 10): [(256, 256), (256, 256), (256, 256), (256, 256), (256, 256), (256, 256), (256, 256), (256, 256), (256, 256), (256, 256)]
📊 Tryby kolorów: Counter({'RGB': 100})


In [ ]:
# 📐 Statystyki rozdzielczości
if dims:
    widths, heights = zip(*dims)
    print(f"\n📏 Rozmiar obrazów — szerokość: {min(widths)}–{max(widths)}, wysokość: {min(heights)}–{max(heights)}")

In [ ]:
import os
import hashlib
from PIL import Image
import numpy as np
import cv2
from tqdm import tqdm
import matplotlib.pyplot as plt
from collections import Counter, defaultdict

In [ ]:
# sprawdzenie przecieków danych między zbiorami
def file_hash(path, block_size=65536):
    hasher = hashlib.md5()
    with open(path, "rb") as f:
        buf = f.read(block_size)
        while buf:
            hasher.update(buf)
            buf = f.read(block_size)
    return hasher.hexdigest()


def collect_hashes(root_dir):
    hashes = defaultdict(list)

    for split in ["train", "valid", "test"]:
        split_path = os.path.join(root_dir, split)
        if not os.path.isdir(split_path):
            continue

        for cls in os.listdir(split_path):
            cls_path = os.path.join(split_path, cls)
            if not os.path.isdir(cls_path):
                continue

            for fname in os.listdir(cls_path):
                if fname.lower().endswith((".jpg", ".png", ".jpeg")):
                    path = os.path.join(cls_path, fname)
                    try:
                        h = file_hash(path)
                        hashes[h].append((split, cls, path))
                    except:
                        pass
    return hashes


hashes = collect_hashes(root_dir)

leaks = {h: v for h, v in hashes.items() if len(v) > 1}

print(f"🔍 Liczba potencjalnych przecieków danych: {len(leaks)}")

for h, locations in list(leaks.items())[:5]:
    print("\n⚠️ Ten sam obraz w:")
    for loc in locations:
        print(" ", loc)

In [ ]:
# analiza trybów kolorów i liczby kanałów
def analyze_color_modes(root_dir, sample_limit=5000):
    modes = Counter()
    channel_counts = Counter()

    files_checked = 0

    for split in ["train", "valid", "test"]:
        for cls in os.listdir(os.path.join(root_dir, split)):
            cls_path = os.path.join(root_dir, split, cls)
            if not os.path.isdir(cls_path):
                continue

            for fname in os.listdir(cls_path):
                if files_checked >= sample_limit:
                    break
                if fname.lower().endswith((".jpg", ".png", ".jpeg")):
                    path = os.path.join(cls_path, fname)
                    try:
                        img = Image.open(path)
                        modes[img.mode] += 1
                        channel_counts[len(img.getbands())] += 1
                        files_checked += 1
                    except:
                        pass

    return modes, channel_counts


modes, channel_counts = analyze_color_modes(root_dir)

print("🎨 Tryby kolorów:", modes)
print("📡 Liczba kanałów:", channel_counts)

In [ ]:
# sprawdzenie uszkodzonych plików obrazów
def find_corrupted_images(root_dir):
    corrupted = []

    for split in ["train", "valid", "test"]:
        for cls in os.listdir(os.path.join(root_dir, split)):
            cls_path = os.path.join(root_dir, split, cls)
            if not os.path.isdir(cls_path):
                continue

            for fname in os.listdir(cls_path):
                if fname.lower().endswith((".jpg", ".png", ".jpeg")):
                    path = os.path.join(cls_path, fname)
                    try:
                        img = Image.open(path)
                        img.verify()
                    except:
                        corrupted.append(path)
    return corrupted


corrupted = find_corrupted_images(root_dir)
print(f"❌ Uszkodzone pliki: {len(corrupted)}")

In [ ]:
# wykrywanie jednokolorowych lub pustych obrazów
def detect_blank_images(root_dir, threshold=5):
    blank = []

    for split in ["train", "valid", "test"]:
        for cls in os.listdir(os.path.join(root_dir, split)):
            cls_path = os.path.join(root_dir, split, cls)
            if not os.path.isdir(cls_path):
                continue

            for fname in os.listdir(cls_path):
                if fname.lower().endswith((".jpg", ".png", ".jpeg")):
                    path = os.path.join(cls_path, fname)
                    try:
                        img = np.array(Image.open(path).convert("RGB"))
                        if img.std() < threshold:
                            blank.append(path)
                    except:
                        pass
    return blank


blank_images = detect_blank_images(root_dir)
print(f"⬜ Jednokolorowe / puste obrazy: {len(blank_images)}")

In [ ]:
# ekstremalne rozmycie
def detect_blurry_images(root_dir, blur_threshold=20.0):
    blurry = []

    for split in ["train", "valid", "test"]:
        for cls in os.listdir(os.path.join(root_dir, split)):
            cls_path = os.path.join(root_dir, split, cls)
            if not os.path.isdir(cls_path):
                continue

            for fname in os.listdir(cls_path):
                if fname.lower().endswith((".jpg", ".png", ".jpeg")):
                    path = os.path.join(cls_path, fname)
                    try:
                        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
                        lap_var = cv2.Laplacian(img, cv2.CV_64F).var()
                        if lap_var < blur_threshold:
                            blurry.append((path, lap_var))
                    except:
                        pass
    return blurry


blurry_images = detect_blurry_images(root_dir)
print(f"🌫️ Ekstremalnie rozmyte obrazy: {len(blurry_images)}")

In [ ]:
# czy są artefakty kompresji:
def detect_compression_artifacts(root_dir, threshold=40):
    artifacts = []

    for split in ["train", "valid", "test"]:
        for cls in os.listdir(os.path.join(root_dir, split)):
            cls_path = os.path.join(root_dir, split, cls)
            if not os.path.isdir(cls_path):
                continue

            for fname in os.listdir(cls_path):
                if fname.lower().endswith(".jpg"):
                    path = os.path.join(cls_path, fname)
                    try:
                        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
                        dct = cv2.dct(np.float32(img))
                        high_freq_energy = np.mean(np.abs(dct[img.shape[0]//2:, img.shape[1]//2:]))
                        if high_freq_energy > threshold:
                            artifacts.append(path)
                    except:
                        pass
    return artifacts


artifact_images = detect_compression_artifacts(root_dir)
print(f"📦 Podejrzenie artefaktów kompresji: {len(artifact_images)}")

In [ ]:
# statystyki pikseli
def compute_mean_std(root_dir, max_images=10000):
    pixels = []
    count = 0

    for split in ["train"]:
        for cls in os.listdir(os.path.join(root_dir, split)):
            cls_path = os.path.join(root_dir, split, cls)
            for fname in os.listdir(cls_path):
                if count >= max_images:
                    break
                if fname.lower().endswith((".jpg", ".png", ".jpeg")):
                    path = os.path.join(cls_path, fname)
                    try:
                        img = np.array(Image.open(path).convert("RGB")) / 255.0
                        pixels.append(img.reshape(-1, 3))
                        count += 1
                    except:
                        pass

    pixels = np.vstack(pixels)
    return pixels.mean(axis=0), pixels.std(axis=0)


mean, std = compute_mean_std(root_dir)

print("📊 Średnia RGB:", mean)
print("📊 Std RGB:", std)

In [ ]:
# histogramy RGB
def plot_rgb_histogram(root_dir, samples=2000):
    colors = {"R": [], "G": [], "B": []}
    count = 0

    for split in ["train"]:
        for cls in os.listdir(os.path.join(root_dir, split)):
            cls_path = os.path.join(root_dir, split, cls)
            for fname in os.listdir(cls_path):
                if count >= samples:
                    break
                if fname.lower().endswith((".jpg", ".png", ".jpeg")):
                    path = os.path.join(cls_path, fname)
                    try:
                        img = np.array(Image.open(path).convert("RGB"))
                        colors["R"].extend(img[:, :, 0].flatten())
                        colors["G"].extend(img[:, :, 1].flatten())
                        colors["B"].extend(img[:, :, 2].flatten())
                        count += 1
                    except:
                        pass

    plt.figure(figsize=(10,5))
    for c in colors:
        plt.hist(colors[c], bins=256, alpha=0.5, label=c)
    plt.legend()
    plt.title("Histogram RGB")
    plt.show()


plot_rgb_histogram(root_dir)

In [ ]:
# czy normalizacja ma sens
imagenet_mean = np.array([0.485, 0.456, 0.406])
imagenet_std  = np.array([0.229, 0.224, 0.225])

print("🔍 Różnica mean:", mean - imagenet_mean)
print("🔍 Różnica std :", std - imagenet_std)